# Miscellaneous of code

### Preprocessing data

In [ ]:
# Define the input shape of the images
img_rows, img_cols = 28, 28

In [ ]:
# Reshape the images to add the mono channel dimension (1)
train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)

In [ ]:
# Convert class vectors to binary class matrices
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
# Save the train and test labels in an argmax variable
train_labels_argmax = np.argmax(train_labels, axis=1, out=None)
test_labels_argmax = np.argmax(test_labels, axis=1, out=None)

### Models

In [ ]:
def create_cnn_model(input_shape, num_classes):
    # Input layer
    inputs = Input(shape=input_shape)
    # Convolutional layers
    x = Conv2D(16, kernel_size=(3, 3), activation=LeakyReLU(0.1))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(32, (3, 3), activation=LeakyReLU(0.1))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # Dropout layer
    x = Dropout(0.25)(x)
    # Flatten layer
    x = Flatten()(x)
    # Dense layers
    x = Dense(64, activation=LeakyReLU(0.1))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
    
model = create_cnn_model((img_rows, img_cols, 1), num_classes)

In [ ]:
# Define the CNN model
model = models.Sequential()

# Convolutional and pooling layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten layer to transition from convolutional to dense layers
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # 10 output classes for Fashion MNIST

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

### Adversarial pattern creation

In [ ]:
# Choose image and label with the model prediction
image_index = 1
image = (np.expand_dims(test_images[image_index], 0))
image = tf.convert_to_tensor(image, dtype=tf.float32)
image_probs = probability_model.predict(image)
print(f"Predicted Class: {np.argmax(image_probs)}")

label = train_labels[image_index]
label = tf.one_hot(image_index, image_probs.shape[-1])
label = tf.reshape(label, (1, image_probs.shape[-1]))

# Create the perturbation pattern
perturbations = create_adversarial_pattern(image, label)
mpl.imshow(perturbations[0] * 0.5 + 0.5)  # To change [-1, 1] to [0,1]